# Module hóa

In [ ]:
# Module hóa
def add_is_night(df):
    df['Is_Night'] = ((df['Hour'] >= 0) & (df['Hour'] <= 5)).astype(int)
    print("Đã thêm đặc trưng Is_Night vào Data")
    print("-"*100)
    return df

def add_High_Amount(df):
      # df = add_High_Amount(df)
    df['High_Amount'] = (df['Amount'] > 1000).astype(int)
    print("Đã thêm đặc trưng High_Amount vào Data")
    print("-"*100)
    return df

def add_Amount_Bins(df):
    bins = [0, 100, 500, 1000, 5000, 2000000]
    labels = [0, 1, 2, 3, 4]
    df['Amount_Bins'] = pd.cut(df['Amount'], bins = bins, labels = labels, include_lowest=True)
    df['Amount_Bins'] = df['Amount_Bins'].astype(int)
    print("Đã thêm đặc trưng Amount_Bins vào Data")
    print("-"*100)
    return df

def add_High_Amount_at_Night(df):
    df['High_Amount_at_Night'] = (df['High_Amount'] & df['Is_Night']).astype(int)
    print("Đã thêm đặc trưng High_Amount_at_Night vào Data")
    print("-"*100)

    return df

def analize_binary_feature(df, col, target = 'Class', Viz = False):
    print(f"Phân tích đặc trưng {col}:")
    print("-"*100)
    print(df[col].value_counts())
    dataa = df.groupby(col)[target].mean()
    print(dataa)

    if Viz:
        print(f"Trực quan hóa cột {col}:")
        print("-"*100)

        fig1 = plt.figure(figsize = (14, 8))

        # Biểu đồ thứ nhất
        plt.subplot(1,2,1)
        sns.countplot(data = df, x = col, hue = target)
        plt.yscale('log')
        plt.title(f'Phân tích Fraud - {col}')
        plt.xlabel(col)
        plt.ylabel('Tần suất xuất hiện (Log Scale)')

        # Biểu đồ thứ hai
        plt.subplot(1,2,2)
        # Phương pháp 2: Chuyển Series thành DataFrame và vẽ bằng seaborn
        dataa_df = dataa.reset_index()  # Chuyển Series thành DataFrame
        dataa_df.columns = [col, target]  # Đặt tên cho các cột

        sns.barplot(data = dataa_df, x= col, y= target, hue = col)
        plt.title(f'Giá trị trung bình của {target} theo {col}')
        plt.ylabel(f'Giá trị trung bình của {target}')

        fig1.suptitle(f"BIỂU ĐỒ PHÂN TÍCH BIẾN {col}", fontsize = 25, color = 'red')
        plt.tight_layout()

        plt.show()
        print("-"*100)

# Preprocess

In [ ]:
# 3 Tiền xử lý dữ liệu
def preprocess_data(df):
    print("\n TIỀN XỬ LÝ DỮ LIỆU:")
    print("-"*100)

    #3.1 Thêm đặc trưng phi tuyến
    print("Tiến hành thêm đặc trưng phi tuyến:")
    print("-"*100)

    # Thêm đặc trưng Is_Night
    df = add_is_night(df)
    analize_binary_feature(df, col = 'Is_Night', Viz = True)

    # Thêm đặc trưng High_Amount
    df = add_High_Amount(df)
    analize_binary_feature(df, col = 'High_Amount', Viz = True)

    # Thêm đặc trưng Amount_Bins
    df = add_Amount_Bins(df)
    analize_binary_feature(df, col = 'Amount_Bins', Viz = True)

    # Thêm đặc trưng High_Amount_at_Night
    df = add_High_Amount_at_Night(df)
    analize_binary_feature(df, col = 'High_Amount_at_Night', Viz = True)

    #3.2 Tách biến đặc trưng và mục tiêu
    print("Tiến hành tách biến đặc trưng và mục tiêu!")
    X = df.drop(['Class', 'Time'], axis = 1)
    y = df['Class']
    print("-"*100)

    #3.3 Chia tập huấn luyện
    print("Tiến hành chia tập huấn luyện!\n")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

    # Kiểm tra lại phân phối nhãn trong tập Train và Test
    print("Tỷ lệ nhãn Fraud trong tập train: ", Counter(y_train)[1]/len(X_train))
    print("Tỷ lệ nhãn Fraud trong tập test: ", Counter(y_test)[1]/len(X_test))

    # Chuẩn hóa: đối với Amount
    # robust_scaler = RobustScaler()
    # X_train['Amount'] = robust_scaler.fit_transform(X_train['Amount'].values.reshape(-1, 1))
    # X_test['Amount'] = robust_scaler.transform(X_test['Amount'].values.reshape(-1, 1))

    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = preprocess_data(df)